In [334]:
import nba_api
import pandas as pd
import time
import pickle
from pprint import pprint

In [377]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog

In [341]:
# Get list of active NBA players
df_activePlayers = pd.DataFrame(players.get_active_players())
df_activePlayers['full_name'] = df_activePlayers['full_name'].apply(lambda x: x.lower())
pprint(df_activePlayers.iloc[1])
print("There are {} active players in the NBA.".format(df_activePlayers['id'].nunique()))

id                  203500
full_name     steven adams
first_name          Steven
last_name            Adams
is_active             True
Name: 1, dtype: object
There are 531 active players in the NBA.


In [379]:
# Test API functions with Anthony Davis Data
AD_info = players.find_players_by_full_name("Anthony Davis")[0]
careerAD = nba_api.stats.endpoints.playercareerstats.PlayerCareerStats(player_id="203076")
careerAD.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203076,2012-13,00,1610612740,NOH,20.0,64,60,1846.0,349,676,0.516,0,6,0.000,169,225,0.751,165,357,522,63,75,112,89,158,867
1,203076,2013-14,00,1610612740,NOP,21.0,67,66,2358.0,522,1005,0.519,2,9,0.222,348,440,0.791,207,466,673,105,89,189,109,200,1394
2,203076,2014-15,00,1610612740,NOP,22.0,68,68,2455.0,642,1199,0.535,1,12,0.083,371,461,0.805,173,523,696,149,100,200,95,141,1656
3,203076,2015-16,00,1610612740,NOP,23.0,61,61,2164.0,560,1136,0.493,35,108,0.324,326,430,0.758,130,497,627,116,78,125,121,148,1481
4,203076,2016-17,00,1610612740,NOP,24.0,75,75,2708.0,770,1526,0.505,40,134,0.299,519,647,0.802,172,712,884,157,94,167,181,168,2099
5,203076,2017-18,00,1610612740,NOP,25.0,75,75,2727.0,780,1462,0.534,55,162,0.340,495,598,0.828,187,644,831,174,115,193,162,159,2110
6,203076,2018-19,00,1610612740,NOP,26.0,56,56,1850.0,530,1026,0.517,48,145,0.331,344,433,0.794,174,498,672,218,88,135,112,132,1452
7,203076,2019-20,00,1610612747,LAL,27.0,62,62,2131.0,551,1096,0.503,72,218,0.330,444,525,0.846,142,435,577,200,91,143,154,156,1618
8,203076,2020-21,00,1610612747,LAL,28.0,36,36,1162.0,301,613,0.491,26,100,0.260,158,214,0.738,62,224,286,110,45,59,74,60,786
9,203076,2021-22,00,1610612747,LAL,29.0,40,40,1404.0,370,695,0.532,13,70,0.186,174,244,0.713,106,288,394,122,49,90,82,97,927


In [98]:
gamelogAD = playergamelog.PlayerGameLog(player_id="203076", season='2016-17')
gamelog_df = gamelogAD.get_data_frames()[0]
print(gamelog_df.iloc[51])
gamelog_df.shape

SEASON_ID                 22016
Player_ID                203076
Game_ID              0021600371
GAME_DATE          DEC 13, 2016
MATCHUP             NOP vs. GSW
WL                            L
MIN                          36
FGM                          10
FGA                          14
FG_PCT                    0.714
FG3M                          1
FG3A                          1
FG3_PCT                     1.0
FTM                           7
FTA                           8
FT_PCT                    0.875
OREB                          0
DREB                          8
REB                           8
AST                           3
STL                           2
BLK                           5
TOV                           6
PF                            2
PTS                          28
PLUS_MINUS                   -8
VIDEO_AVAILABLE               1
Name: 51, dtype: object


(75, 27)

In [296]:
# Anthony Davis
gamelogAD = playergamelog.PlayerGameLog(player_id="203076", season='2016-17')
df_gamelogAD = gamelogAD.get_data_frames()[0]

df_gamelogAD['MATCHUP'] = df_gamelogAD['MATCHUP'].apply(lambda x : x.split())
df_gamelogAD['HOME'] = df_gamelogAD['MATCHUP'].apply(lambda x : 1 if x[1] == 'vs.' else 0)
df_gamelogAD['OPP'] = df_gamelogAD['MATCHUP'].apply(lambda x : x[2])
df_gamelogAD['TEAM'] = df_gamelogAD['MATCHUP'][0][0]
df_gamelogAD['GAME_DATE'] = pd.to_datetime(df_gamelogAD['GAME_DATE'], format='%b %d, %Y')

removeCols = ['Game_ID', 'MATCHUP', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'VIDEO_AVAILABLE']
df_gamelogAD.drop(columns= removeCols, inplace=True)
#print(df_gamelogAD.iloc[51])
print(df_gamelogAD.shape)
df_gamelogAD.head(10)

(75, 24)


,SEASON_ID,Player_ID,GAME_DATE,WL,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,HOME,OPP,TEAM
0,22016,203076,2017-04-07,L,24,11,27,0,3,3,5,2,4,6,0,1,0,1,2,25,-2,0,DEN,NOP
1,22016,203076,2017-04-04,L,41,18,31,2,5,3,3,2,3,5,0,1,4,0,3,41,9,1,DEN,NOP
2,22016,203076,2017-04-02,L,40,8,19,1,4,13,14,2,9,11,2,3,1,3,2,30,-6,1,CHI,NOP
3,22016,203076,2017-03-31,W,34,9,19,0,2,1,1,4,8,12,2,1,6,1,2,19,25,1,SAC,NOP
4,22016,203076,2017-03-29,W,36,11,22,0,1,8,8,4,10,14,2,2,1,1,1,30,1,1,DAL,NOP
5,22016,203076,2017-03-27,L,41,12,23,1,2,11,14,1,16,17,3,3,3,1,1,36,0,0,UTA,NOP
6,22016,203076,2017-03-26,W,28,12,17,1,2,6,7,1,12,13,3,0,1,3,3,31,20,0,DEN,NOP
7,22016,203076,2017-03-24,L,41,14,24,0,1,5,6,3,13,16,1,3,1,1,2,33,-9,0,HOU,NOP
8,22016,203076,2017-03-21,W,40,9,18,0,2,1,2,3,10,13,0,0,0,2,2,19,24,1,MEM,NOP
9,22016,203076,2017-03-19,W,35,11,16,0,2,6,6,4,7,11,3,0,0,5,1,28,16,1,MIN,NOP


In [144]:
seasonYears = ["20{}-{}".format(16+i, 16+i+1) for i in range(8)]
print(seasonYears)

['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']


In [371]:
removeCols = ['Game_ID', 'MATCHUP', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'VIDEO_AVAILABLE']

def getCareerLog(player_id):
    career = []

    for season in seasonYears:
        try:
            gamelog = playergamelog.PlayerGameLog(player_id=player_id, season=season)
        except:
            print("Invalid Entry. Player stats not found.")
            break
            
        df_gamelog = gamelog.get_data_frames()[0]

        if len(df_gamelog) > 1:
            df_gamelog['MATCHUP'] = df_gamelog['MATCHUP'].apply(lambda x : x.split())
            df_gamelog['HOME'] = df_gamelog['MATCHUP'].apply(lambda x : 1 if x[1] == 'vs.' else 0)
            df_gamelog['OPP'] = df_gamelog['MATCHUP'].apply(lambda x : x[2])
            df_gamelog['TEAM'] = df_gamelog['MATCHUP'][0][0]
            df_gamelog['GAME_DATE'] = pd.to_datetime(df_gamelog['GAME_DATE'], format='%b %d, %Y')        
            df_gamelog.drop(columns= removeCols, inplace=True)
            df_gamelog = df_gamelog[::-1]   # reverse game order
            df_gamelog = df_gamelog.reset_index(drop=True)
            career.append(df_gamelog)
            time.sleep(.600)
    
    # Combine all season into one consecutive data frame
    df_career = pd.concat(career) if len(career) > 1 else None

    return df_career

In [255]:
lamelo_info = players.find_players_by_full_name("LaMelo Ball")[0]
print(lamelo_info)
gamelogLB = playergamelog.PlayerGameLog(player_id=lamelo_info['id'], season=seasonYears[0])
df_lamelo_career = getCareerLog(lamelo_info['id'])
#df = gamelogLB.get_data_frames()[0]
print(df_lamelo_career.shape)
df_lamelo_career.head(10)

{'id': 1630163, 'full_name': 'LaMelo Ball', 'first_name': 'LaMelo', 'last_name': 'Ball', 'is_active': True}
(184, 24)


,SEASON_ID,Player_ID,GAME_DATE,WL,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,HOME,OPP,TEAM
0,22020,1630163,2020-12-23,L,16,0,5,0,3,0,0,0,1,1,3,2,0,3,2,0,2,0,CLE,CHA
1,22020,1630163,2020-12-26,L,15,5,10,2,4,1,2,2,3,5,1,1,0,2,1,13,-11,1,OKC,CHA
2,22020,1630163,2020-12-27,W,20,2,6,2,3,0,0,0,5,5,5,1,0,3,4,6,-4,1,BKN,CHA
3,22020,1630163,2020-12-30,W,29,7,10,4,5,4,4,4,4,8,5,1,1,2,3,22,8,0,DAL,CHA
4,22020,1630163,2021-01-01,L,27,6,14,2,7,1,2,1,2,3,6,2,0,2,2,15,0,1,MEM,CHA
5,22020,1630163,2021-01-02,L,31,4,12,1,4,4,6,2,2,4,5,4,2,2,4,13,-10,0,PHI,CHA
6,22020,1630163,2021-01-04,L,30,5,16,0,5,2,3,2,5,7,9,2,0,2,2,12,-9,0,PHI,CHA
7,22020,1630163,2021-01-06,W,24,5,11,1,4,2,2,2,6,8,5,0,0,1,2,13,0,0,ATL,CHA
8,22020,1630163,2021-01-08,W,26,4,11,1,5,3,4,4,6,10,9,1,0,3,3,12,5,0,NOP,CHA
9,22020,1630163,2021-01-09,W,31,9,13,3,5,1,2,3,9,12,11,1,0,1,4,22,18,1,ATL,CHA


In [ ]:
# Create dictionary to store dataframe of each players career game log
activePlayerCareerLog = {}

In [333]:
# Collect all active player data from NBA API

for index in df_activePlayers.index:
    full_name = df_activePlayers['full_name'][index]
    if full_name in activePlayerCareerLog:
        continue

    career_log = getCareerLog(df_activePlayers['id'][index])
    
    if not isinstance(career_log, pd.DataFrame):
        continue

    activePlayerCareerLog[full_name] = career_log
    print("Collected stats for: {}".format(full_name))
    time.sleep(5)   # Add delay to prevent API timeout 

Collected stats for: davis bertans
Collected stats for: patrick beverley
Collected stats for: saddiq bey
Collected stats for: goga bitadze
Collected stats for: bismack biyombo
Collected stats for: bogdan bogdanovic
Collected stats for: bojan bogdanovic
Collected stats for: bol bol
Collected stats for: marques bolden
Collected stats for: devin booker
Collected stats for: brandon boston jr.
Collected stats for: chris boucher
Collected stats for: james bouknight
Collected stats for: malaki branham
Collected stats for: christian braun
Collected stats for: mikal bridges
Collected stats for: miles bridges
Collected stats for: oshae brissett
Collected stats for: malcolm brogdon
Collected stats for: armoni brooks
Collected stats for: dillon brooks
Collected stats for: bruce brown
Collected stats for: jaylen brown
Collected stats for: kendall brown
Collected stats for: moses brown
Collected stats for: greg brown iii
Collected stats for: charlie brown jr.
Collected stats for: troy brown jr.
Coll

In [ ]:
# Remove any empty values from player careers
emptyKeys = [k for k, v in activePlayerCareerLog.items() if not isinstance(v, pd.DataFrame)]
for k in emptyKeys:
    del activePlayerCareerLog[k]

In [374]:
# Save collected player data into a pickle file
file_path = './Data/RAW_activePlayerCareerLogData.pickle'
with open(file_path, 'wb') as f:
    pickle.dump(activePlayerCareerLog, f)

In [375]:
# Load collected player data from pickle file
with open(file_path, 'rb') as file:
    loaded_careerLog = pickle.load(file)

In [376]:
loaded_careerLog['zach lavine']

,SEASON_ID,Player_ID,GAME_DATE,WL,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,HOME,OPP,TEAM
0,22016,203897,2016-10-26,L,34,6,14,3,6,4,5,0,5,5,3,1,0,2,3,19,-4,0,MEM,MIN
1,22016,203897,2016-10-29,L,37,7,14,3,7,4,4,0,5,5,2,0,0,1,3,21,-3,0,SAC,MIN
2,22016,203897,2016-11-01,W,29,11,18,5,9,4,4,0,4,4,2,0,0,3,2,31,28,1,MEM,MIN
3,22016,203897,2016-11-03,L,36,4,13,2,6,4,6,1,4,5,3,1,0,0,2,14,-2,1,DEN,MIN
4,22016,203897,2016-11-05,L,26,1,6,1,3,1,1,1,1,2,2,1,0,2,0,4,-24,0,OKC,MIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,22023,203897,2024-01-10,W,41,8,15,4,8,5,6,1,12,13,7,3,1,4,5,25,15,1,HOU,CHI
21,22023,203897,2024-01-12,L,35,9,15,4,8,3,3,0,8,8,7,0,1,1,3,25,2,1,GSW,CHI
22,22023,203897,2024-01-13,W,36,3,8,1,5,3,4,0,6,6,7,1,0,2,3,10,12,0,SAS,CHI
23,22023,203897,2024-01-15,L,35,7,14,3,6,0,0,0,4,4,4,0,0,3,3,17,-18,0,CLE,CHI
